In [9]:
from flask import Flask, request, jsonify, render_template, send_file
import joblib
import pandas as pd
from werkzeug.utils import secure_filename
import os
import threading

app = Flask(__name__)

# Load the model and vectorizer
model = joblib.load('model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

# Define a directory to save uploaded files
UPLOAD_FOLDER = 'uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/')
def home():
    return render_templates('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' in request.files:
        file = request.files['file']
        if file.filename == '':
            return "No selected file", 400

        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        # Read the CSV file
        data = pd.read_csv(filepath)
        
        # Ensure the text column does not contain NaN values
        if 'text' not in data.columns:
            return "No 'text' column found in the uploaded file", 400
        data['text'] = data['text'].fillna('')

        # Vectorize the data
        data_vectorized = vectorizer.transform(data['text'])
        
        # Make predictions
        predictions = model.predict(data_vectorized)
        
        # Map predictions to sentiments
        data['sentiment'] = predictions
        
        # Save the predictions to a new CSV file
        predictions_filepath = os.path.join(app.config['UPLOAD_FOLDER'], 'predictions.csv')
        data.to_csv(predictions_filepath, index=False)
        
        # Send the file as response
        return send_file(predictions_filepath, as_attachment=True, download_name='predictions.csv')

    elif request.json and 'text' in request.json:
        text = request.json['text']
        
        # Vectorize the input text
        input_data_vectorized = vectorizer.transform([text])
        
        # Make prediction
        prediction = model.predict(input_data_vectorized)[0]
        
        return jsonify({'prediction': prediction})

    return "No file or text input provided", 400

def run_app():
    app.run(debug=True, use_reloader=False)

# Run the Flask app in a separate thread to avoid blocking the Jupyter Notebook
if __name__ == '__main__':
    threading.Thread(target=run_app).start()


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Jun/2024 08:23:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2024 08:23:45] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2024 08:23:52] "POST /predict HTTP/1.1" 200 -
